# Memory & Threads in Conversational Agentic AI Systems in LangGraph

Every interaction between the user and the agentic system is in isolation and a new request without keep track of past historical conversations (by default). By leveraging memory and threads we can make our agentic system conversational.

![](https://i.imgur.com/nJn1o09.png)

#### Memory in LangGraph:

- Utilizes a built-in persistence layer to maintain graph state across executions.
- Enables features like human-in-the-loop interactions, time travel, fault tolerance and conversational capabilities

#### Threads:

- Serve as unique identifiers for sequences of checkpoints (agent state snapshots).
- Allow retrieval and management of graph states post-execution.
- Specified during graph invocation via {"configurable": {"thread_id": "user-session-id"}}.
- User session id can be generated and assigned per unique user or user session
- This is used by the agent to refer to past conversation and agent state history for any user session at any time
- Enables multi-user conversation for your agent


In [0]:
!pip install langchain==0.3.14
!pip install langchain-openai==0.3.0
!pip install langchain-community==0.3.14
!pip install langgraph==0.2.64
!pip install langgraph-checkpoint-sqlite==2.0.3

## Enter Open AI API Key

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [0]:
TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

## Setup Environment Variables

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## State

First, define the [State](https://langchain-ai.github.io/langgraph/concepts/low_level/#state) of the graph.

The State schema serves as the input schema for all Nodes and Edges in the graph.

Let's use the `TypedDict` class from python's `typing` module as our schema, which provides type hints for the keys.

In [0]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

## Augment the LLM with tools

Here we define our custom search tool and then bind it to the LLM to augment the LLM

In [0]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.tools import tool

llm = ChatOpenAI(model="gpt-4o", temperature=0)

tavily_search = TavilySearchAPIWrapper()
@tool
def search_web(query: str, num_results=5):
    """Search the web for a query. Userful for general information or general news"""
    results = tavily_search.raw_results(query=query,
                                        max_results=num_results,
                                        search_depth='advanced',
                                        include_raw_content=True)
    return results

tools = [search_web]
llm_with_tools = llm.bind_tools(tools=tools)

## Build Agentic Graph with In-Memory Persistence

In [0]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


# Augmented LLM with Tools Node function
def tool_calling_llm(state: State) -> State:
    current_state = state["messages"]
    return {"messages": [llm_with_tools.invoke(current_state)]}

# Build the graph
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools=tools))
builder.add_edge(START, "tool_calling_llm")

# Conditional Edge
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from LLM is a tool call -> tools_condition routes to tools
    # If the latest message (result) from LLM is a not a tool call -> tools_condition routes to END
    tools_condition,
    ["tools", END]
)
builder.add_edge("tools", "tool_calling_llm") # this is the key feedback loop
builder.add_edge("tools", END)

# add in-memory persistence (transient memory)
memory = MemorySaver()
agent_inmem = builder.compile(checkpointer=memory)

In [0]:
agent_inmem

## Test Agent with In-Memory Persistence

In [0]:
uid = 'user001'
config = {"configurable": {"thread_id": uid}}

In [0]:
user_input = "Explain AI in 1 line"
for event in agent_inmem.stream(input={"messages": user_input},
                          config=config,
                          stream_mode='values'):
    event['messages'][-1].pretty_print()

In [0]:
user_input = "Do the same for ML"
for event in agent_inmem.stream(input={"messages": user_input},
                          config=config,
                          stream_mode='values'):
    event['messages'][-1].pretty_print()

In [0]:
agent_inmem.get_state(config)

In [0]:
history = list(agent_inmem.get_state_history(config))
history

In [0]:
uid = 'user002'
config = {"configurable": {"thread_id": uid}}

In [0]:
user_input = "Tell me 3 latest OpenAI product releases"
for event in agent_inmem.stream(input={"messages": user_input},
                          config=config,
                          stream_mode='values'):
    event['messages'][-1].pretty_print()

In [0]:
user_input = "do the same for Meta releases"
for event in agent_inmem.stream(input={"messages": user_input},
                          config=config,
                          stream_mode='values'):
    event['messages'][-1].pretty_print()

In [0]:
uid = 'user001'
config = {"configurable": {"thread_id": uid}}
user_input = "what did we discuss so far"
for event in agent_inmem.stream(input={"messages": user_input},
                          config=config,
                          stream_mode='values'):
    event['messages'][-1].pretty_print()

## Build & Test Agentic Graph with On-disk Persistence

In [0]:
# clearing memory database (just for demo, in general should keep it)
!rm memory.db*

In [0]:
from langgraph.checkpoint.sqlite import SqliteSaver

def call_conversational_agent(agent_graph, prompt, user_session_id):
    with SqliteSaver.from_conn_string("memory.db") as memory:
        agent_extmem = agent_graph.compile(checkpointer=memory)
        for event in agent_extmem.stream(input={"messages": prompt},
                                         config={"configurable": {"thread_id": user_session_id}},
                                         stream_mode='values'):
            event['messages'][-1].pretty_print()

In [0]:
builder

In [0]:
uid = 'bond007'
prompt = "What is the latest news on Apple? summarize in 3 bullets"
call_conversational_agent(agent_graph=builder,
                          prompt=prompt,
                          user_session_id=uid)

In [0]:
prompt = "What about microsoft?"
call_conversational_agent(agent_graph=builder,
                          prompt=prompt,
                          user_session_id=uid)